In [13]:
import os
import sys

diretorio_atual = os.getcwd()
print(f"Logado no diretório: {diretorio_atual}")
sys.path.append('/home/jovyan/work')

Logado no diretório: /home/jovyan/work


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import joblib

experimento = 0

# Função para transformar e escorar uma nova base de dados
def escorar_nova_base(dados, caminho_modelo, caminho_saida):

    # Separar as variáveis explicativas e a variável alvo
    X = dados.copy()
    X = X.drop(columns=['y'], errors='ignore')

    # Transformar variáveis especificadas
    transform_list = ['age', 'duration']
    for col in transform_list:
        if col in X.columns:
            X[col] = X[col].replace([np.inf, -np.inf], np.nan).fillna(0)
            X[f'{col}_log'] = np.log1p(X[col])
            X = X.drop(columns=[col])

    # Remove as colunas que apresentam alta correlação e IV: `usefull`
    colunas_para_remover = ['emp.var.rate', 'euribor3m', 'nr.employed', 'day_of_week', 'housing', 'loan', 'pdays']
    X = X.drop(columns=colunas_para_remover, errors='ignore')

    # Verificar e imputar valores ausentes nas variáveis numéricas
    variaveis_numericas = ['campaign', 'previous', 'cons.price.idx', 'cons.conf.idx', 'age_log', 'duration_log']
    imputer_numerico = SimpleImputer(strategy='mean')
    X[variaveis_numericas] = imputer_numerico.fit_transform(X[variaveis_numericas])

    # Escalar variáveis numéricas
    scaler = joblib.load(f'./model/Experimento_{experimento}_scaler.pkl')
    variaveis_numericas_existentes = [col for col in variaveis_numericas if col in X.columns]
    X_numerico = scaler.transform(X[variaveis_numericas_existentes])

    # Verificar e imputar valores ausentes nas variáveis categóricas
    variaveis_categoricas = ['job', 'marital', 'education', 'default', 'contact', 'month', 'poutcome']
    imputer_categorico = SimpleImputer(strategy='most_frequent')
    X[variaveis_categoricas] = imputer_categorico.fit_transform(X[variaveis_categoricas])

    # Codificar variáveis categóricas
    encoder = joblib.load(f'./model/Experimento_{experimento}_encoder.pkl')
    variaveis_categoricas_existentes = [col for col in variaveis_categoricas if col in X.columns]
    X_categorico = encoder.transform(X[variaveis_categoricas_existentes])

    # Combinar variáveis numéricas e categóricas transformadas
    X_transformado = np.hstack((X_numerico, X_categorico))

    # Carregar o modelo e fazer previsões
    modelo = joblib.load(caminho_modelo)
    y_prob = modelo.predict_proba(X_transformado)[:, 1]

    # Adicionar as probabilidades ao DataFrame original
    df_predict = dados.copy()
    df_predict['y_prob'] = y_prob

    # Remover as colunas transformadas e as colunas removidas
    df_predict.drop(columns=colunas_para_remover, errors='ignore')

    # Salvar o DataFrame com as probabilidades
    df_predict.to_csv(caminho_saida, index=False)
    print("Dados escorados e salvos com sucesso!")

In [11]:
import os

def listar_arquivos_csv(pasta):
    # Obter a lista de arquivos na pasta
    arquivos = os.listdir(pasta)
    
    # Filtrar e retornar apenas os arquivos .csv
    arquivos_csv = [arquivo for arquivo in arquivos if arquivo.endswith('.csv')]
    
    return arquivos_csv

In [12]:
# Caminho da pasta que contém os arquivos .csv
caminho_pasta = './data/Monitoramento/'

# Chamar a função e listar os arquivos .csv
arquivos_csv = listar_arquivos_csv(pasta = caminho_pasta)
arquivos_csv

caminho_modelo = f'./model/Experimento_{experimento}_modelo_lr.pkl'
caminho_saida = './data/Monit_escorado/'

for i in range(len(arquivos_csv)):
    dados = pd.read_csv(caminho_pasta + arquivos_csv[i])
    caminho_saida = f'./data/Monit_escorado/{arquivos_csv[i][:10]}.csv'
    escorar_nova_base(dados= dados, caminho_modelo = caminho_modelo, caminho_saida = caminho_saida)


/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Dados escorados e salvos com sucesso!


/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Dados escorados e salvos com sucesso!


/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


...

Dados escorados e salvos com sucesso!


/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Dados escorados e salvos com sucesso!


/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Dados escorados e salvos com sucesso!
